In [1]:
from xgboost import XGBClassifier
import warnings
from tabpfn import TabPFNClassifier
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from data_prep_utils import *
from evaluate import *
from load_models import *
import matplotlib.pyplot as plt
import torch
import openml
import time

In [2]:
path = "datasets/data_all.csv"
data, labels = get_microbiome(path)
data = top_non_zero(data)
data, labels = unison_shuffled_copies(data, labels)

In [3]:
for sampling in [oversample, undersample, None]:
    cv = 3
    strat_split = True
    n_optim = 1000
    ft_epochs = 10
    metrics = metrics = ["accuracy", "precision", "recall", "roc_auc"]
    models = [
        XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic'),
        XGBoostOptim(n_optim=n_optim),
        LogisticRegression(max_iter=500), 
        TabPFNClassifier(device='cpu', N_ensemble_configurations=3),
        TabForestPFNClassifier("saved_models/tabforest/mix600k/tabforestpfn.pt", "saved_models/tabforest/mix600k/config_run.yaml", max_epochs=ft_epochs)
    ]
    results = pd.DataFrame(np.zeros((len(models), len(metrics)+1)), 
                           index=[m.__class__.__name__ for m in models],
                          columns=metrics+["runtime"])
    
    for ii, model in enumerate(models):
        results.iloc[ii,:] = cross_validate_sample(model, data, labels, metrics, strat_split, cv, sampling)
    print(results)

(14810, 100) (14810,)
(14810, 100) (14810,)
(14810, 100) (14810,)
(14810, 100) (14810,)


[W 2024-09-28 17:37:03,678] Trial 4 failed with parameters: {'booster': 'gbtree', 'lambda': 3.25111713888519e-07, 'alpha': 5.5714424430947225e-06, 'subsample': 0.7198822314818073, 'colsample_bytree': 0.9993302175408134, 'max_depth': 9, 'min_child_weight': 9, 'eta': 0.028462646620719668, 'gamma': 1.1094084901242425e-07, 'grow_policy': 'depthwise'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\lenna\Desktop\MT\TabPFN-medical\load_models.py", line 116, in objective
    model.fit(X_train, y_train)
  File "C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\xgboost\sklearn.py", line 1531, in fit


In [ ]:
def stratified_split(data, labels, cv=3):
    size = labels.shape[0]
    fold_size = size//cv
    counts = np.unique(labels, return_counts=True)
    c0_size = np.floor(fold_size*counts[1][0]/size).astype(int)
    c1_size = np.floor(fold_size*counts[1][1]/size).astype(int)#fold_size-c0_size
    
    c0_data = data[labels==0]
    c1_data = data[labels==1]
    np.random.shuffle(c0_data)
    np.random.shuffle(c1_data)
    
    data_folds, labels_folds = [], []
    for f in range(cv):
        data_single_fold = np.concatenate((c0_data[c0_size*f:c0_size*(f+1),:],c1_data[c1_size*f:c1_size*(f+1),:]))
        labels_single_fold = np.concatenate((np.zeros((c0_size)), np.ones((c1_size))))
        data_single_fold, labels_single_fold = unison_shuffled_copies(data_single_fold, labels_single_fold)
        data_folds.append(data_single_fold)
        labels_folds.append(labels_single_fold)
        
    return data_folds, labels_folds
data[100] = 1e10
data_folds, labels_folds = stratified_split(data, labels)

for fold in labels_folds:
    counts = np.unique(fold, return_counts=True)
    print(counts[1][0]/np.sum(counts[1]))
for fold in data_folds:
    print(np.max(fold))